# Discover The Relation Between Sex Ratio And Weight Of Advanced Subjects Test Grade

## import packages

In [1]:
import numpy as np
import pandas as pd
import tabula
from sklearn.linear_model import LinearRegression

## read table of weight of AST(Advanced Subjects Test) Grade

In [2]:
weightList = tabula.read_pdf("https://www.uac.edu.tw/107data/107_02.pdf", multiple_tables=True, pages = 'all')
newList = []
for df in weightList:
    colnames = []
    for col in df:
        if str(df[col][5]) != "nan":
            colnames.append(col)
    newList.append(df[colnames])
newList = [weight.iloc[3:, [1,2,3]] for weight in newList]
for df in newList:
    df.columns = ["校名", "系組名", "採計"]
weight = pd.concat(newList, axis = 0)

weight["國"] = weight["採計"].apply(lambda x : 0.0 if x.find("國") == -1 else float(x[x.find("國") + 2 : x.find("國") + 6]))
weight["英"] = weight["採計"].apply(lambda x : 0.0 if x.find("英") == -1 else float(x[x.find("英") + 2 : x.find("英") + 6]))
weight["數甲"] = weight["採計"].apply(lambda x : 0.0 if x.find("數甲") == -1 else float(x[x.find("數甲") + 3 : x.find("數甲") + 7]))
weight["數乙"] = weight["採計"].apply(lambda x : 0.0 if x.find("數乙") == -1 else float(x[x.find("數乙") + 3 : x.find("數乙") + 7]))
weight["歷"] = weight["採計"].apply(lambda x : 0.0 if x.find("歷") == -1 else float(x[x.find("歷") + 2 : x.find("歷") + 6]))
weight["地"] = weight["採計"].apply(lambda x : 0.0 if x.find("地") == -1 else float(x[x.find("地") + 2 : x.find("地") + 6]))
weight["公"] = weight["採計"].apply(lambda x : 0.0 if x.find("公") == -1 else float(x[x.find("公") + 2 : x.find("公") + 6]))
weight["物"] = weight["採計"].apply(lambda x : 0.0 if x.find("物") == -1 else float(x[x.find("物") + 2 : x.find("物") + 6]))
weight["化"] = weight["採計"].apply(lambda x : 0.0 if x.find("化") == -1 else float(x[x.find("化") + 2 : x.find("化") + 6]))
weight["生"] = weight["採計"].apply(lambda x : 0.0 if x.find("生") == -1 else float(x[x.find("生") + 2 : x.find("生") + 6]))
weight["術科"] = weight["採計"].apply(lambda x : 0.0 if x.find("術科") == -1 else float(x[x.find("術科") + 3 : x.find("術科") + 7]))

## read the table of sex ratio of university-departments 

In [3]:
gender = pd.read_csv("./107_students.csv")    # the website is under maintenance
gender.columns = gender.loc[1, :]
gender = gender[(gender["日間∕進修別"] == "D 日") & (gender.等級別 == "B 學士") & (gender.總計 != "-")]
gender = gender.loc[2:, ["學校名稱", "科系名稱", "男生計", "女生計"]]
gender.男生計 = gender.男生計.apply(lambda x : float(x.replace(',', '')) if x != "-" else 0)
gender.女生計 = gender.女生計.apply(lambda x : float(x.replace(',', '')) if x != "-" else 0)
gender["性別比"] = gender.男生計 / gender.女生計

## combine the tables

In [5]:
weightGender = []
for gindex, grow in gender.iterrows():
    for windex, wrow in weight.iterrows():
        if grow["學校名稱"] == wrow["校名"] and grow["科系名稱"] == wrow["系組名"]:
            weightGender.append([grow["學校名稱"], grow["科系名稱"], grow["性別比"], *(list(wrow["國" : "術科"]))])
weightGender = pd.DataFrame(weightGender, columns = ["校名", "系名", "性別比", *(list(weight.columns.values[3:]))])

In [12]:
weightGender.head()

,校名,系名,性別比,國,英,數甲,數乙,歷,地,公,物,化,生,術科
0,國立政治大學,教育學系,0.422619,1.25,2.00,0.0,1.00,1.0,0.0,1.00,0.0,0.0,0.0,0.0
1,國立政治大學,歷史學系,1.881356,1.00,1.00,0.0,1.00,1.5,1.0,0.00,0.0,0.0,0.0,0.0
2,國立政治大學,民族學系,0.762376,1.00,1.00,0.0,0.00,0.0,0.0,1.25,0.0,0.0,0.0,0.0
3,國立政治大學,哲學系,1.129412,1.25,1.25,0.0,1.25,1.0,0.0,1.00,0.0,0.0,0.0,0.0
4,國立政治大學,英國語文學系,0.425234,1.50,2.00,0.0,1.00,1.0,1.0,0.00,0.0,0.0,0.0,0.0


## construct regression model

In [10]:
regr = LinearRegression()
regr.fit(weightGender.loc[:, "國":"術科"], weightGender.性別比)
pd.DataFrame(np.array([[*(list(weight.columns.values[3:]))], regr.coef_]))
# Physics is most postitvely related and Biology is most negatively related 

,0,1,2,3,4,5,6,7,8,9,10
0,國,英,數甲,數乙,歷,地,公,物,化,生,術科
1,0.7203516208537517,-0.38497306850780855,0.7322896401571453,-0.005205131533560685,-0.48749279263800527,-0.08690110226620124,-0.3383614659933974,1.4130321379615958,-0.04683830286970279,-0.75451976775098,-0.28100179128870834
